In [ ]:
# 抽取出整个图中的若干个连通分量

edgedict = {}
with open('../graph/adjl.adjlist', 'r') as adjfile:
    i = 0
    for node in adjfile:
        i += 1
        if i <= 3:
            continue
        l = node.split()
        l = [int(x) for x in l]
        edgedict[l[0]] = l[1:]
q = []
v = [0] * 764879
fr = 0
ed = 0
with open('../graph/partition.txt', 'w') as part:  # bfs找连通分量
    for i in range(188685):
        if not v[i]:
            q.append(i)
            v[i] = 1
            ed += 1
            bg = fr
            while fr < ed:
                s = q[fr]
                fr += 1
                if s not in edgedict.keys():
                    print(s)
                    continue
                sedges = edgedict[s]
                for nd in sedges:
                    if not v[nd]:
                        v[nd] = 1
                        q.append(nd)
                        ed += 1
            part.write(str(ed - bg) + '\n')
            for j in range(bg, ed):
                part.write(str(q[j]) + ' ')
            part.write('\n')
    for i in range(400000, 764879):
        if not v[i]:
            q.append(i)
            v[i] = 1
            ed += 1
            bg = fr
            while fr < ed:
                s = q[fr]
                sedges = edgedict[s]
                for nd in sedges:
                    if not v[nd]:
                        v[nd] = 1
                        q.append(nd)
                        ed += 1
                fr += 1
            part.write(str(ed - bg) + '\n')
            for j in range(bg, ed):
                part.write(str(q[j]) + ' ')
            part.write('\n')

In [ ]:
# 抽取出最大的连通分量
with open('graph/partition.txt', 'r') as part:
    with open('graph/connected.txt', 'w') as c:
        c.write(part.readline())
        c.write(part.readline())

In [ ]:
# 抽取出该连通分量的adjlist
with open('../graph/adjl.adjlist', 'r') as wholeadjl:
    with open('../graph/connected.txt', 'r') as con:
        with open('../graph/connected_adjl.adjlist', 'w') as wrfile:
            con.readline()
            nodes = con.readline()
            nodes = nodes.split()
            nodes = [int(x) for x in nodes]
            nodes = set(nodes)
            i = 0
            for line in wholeadjl:
                i += 1
                if i <= 3:
                    continue
                l = line.find(' ')
                if l == -1:
                    x = int(line)
                else:
                    x = int(line[:l])
                if x in nodes:
                    wrfile.write(line)

In [2]:
# adjlist转edgelist
import networkx as nx
file_edgel = '../graph/connected_edgel.edgelist'
file_adjl = '../graph/connected_adjl.adjlist'
with open(file_adjl, 'r', encoding='utf-8') as stat:
    ss = stat.read()
    G = nx.read_adjlist(file_adjl)
nx.write_edgelist(G, file_edgel, data=False)

In [3]:
# 分别记录该连通分量中的仓库和贡献者
with open('../graph/connected.txt', 'r') as con:
    x = int(con.readline())
    nodes = con.readline()
    nodes = nodes.split()
    nodes = [int(x) for x in nodes]
with open('../graph/connected_repos.txt', 'w') as repos:
    with open('../graph/connected_users.txt', 'w') as users:
        for node in nodes:
            if node < 400000:
                repos.write(str(node) + ' ')
            else:
                users.write(str(node) + ' ')

In [5]:
# bfs得一棵生成树
# 为了保证隐藏若干条边后剩下的图仍联通，保留一棵生成树，该树的边不会被选入测试集
edgedict = {}
with open('../graph/connected_adjl.adjlist', 'r') as adjfile:
    i = 0
    for node in adjfile:
        l = node.split()
        l = [int(x) for x in l]
        edgedict[l[0]] = l[1:]
q = []
v = [0] * 764879
fr = 0
ed = 0
with open('../graph/tree_edge.txt', 'w') as part:
    with open('../graph/connected_repos.txt', 'r') as repos_file:
        repos = repos_file.readline()
        repos = repos.split()
        repos = [int(x) for x in repos]
    q.append(0)
    v[0] = 1
    ed = 1
    bg = 0
    while fr < ed:
        s = q[fr]
        fr += 1
        if s not in edgedict.keys():
            continue
        sedges = edgedict[s]

        for nd in sedges:
            if not v[nd]:
                v[nd] = 1
                q.append(nd)
                ed += 1
                if s > nd:
                    part.write(str(nd) + ' ' + str(s) + '\n')
                else:
                    part.write(str(s) + ' ' + str(nd) + '\n')

In [6]:
# 抽取测试集
import random
with open('../graph/connected_edgel.edgelist', 'r') as alledges:
    edgeso = alledges.readlines()
edges = []
for edge in edgeso:
    [x, y] = edge.split()
    x = int(x)
    y = int(y)
    if x > y:
        edges.append(str(y) + ' ' + str(x) + '\n')
    else:
        edges.append(edge)
edgeset = set(edges)
with open('../graph/tree_edge.txt', 'r') as treeedges:
    tree_edges = treeedges.readlines()
treeset = set(tree_edges)
edgeset = edgeset - treeset
edges = list(edgeset)
sz = len(edges)
with open('../graph/test_edgel_con.edgelist', 'w') as test:
    with open('../graph/train_edgel_con.edgelist', 'w') as train:
        chosen = random.sample(range(sz), 60000)  # 共有301103条边，从中抽取约20%即60000条
        s = set(chosen)
        for i in range(sz):
            if i in s:
                test.write(edges[i])
            else:
                train.write(edges[i])
        for edge in tree_edges:
            train.write(edge)

In [7]:
# 生成负样本
import numpy as np
edgedict = {}
with open('../graph/connected_adjl.adjlist', 'r') as adjfile:
    for node in adjfile:
        l = node.split()
        l = [int(x) for x in l]
        edgedict[l[0]] = l[1:]
s = np.random.randint(0, 35085, 301103).tolist()   # 选择源点
si = []
with open('../graph/connected_repos.txt', 'r') as repos:
    rs = repos.readline()
    rs = rs.split()
    rs = [int(x) for x in rs]
    for x in s:
        si.append(rs[x])
with open('../graph/connected_users.txt', 'r') as users:
    us = users.readline()
    us = us.split()
    us = [int(x) for x in us]
chosen_edges = set()
get_edges = []
for repo in si:  # 对选择的每个源点（仓库），随机抽取一个贡献者，若之间无边则为一个负样本
    while True:
        t = random.randint(0, 132938)
        t = us[t]
        if (repo not in edgedict[t]) and (t not in edgedict[repo]) and ((repo, t) not in chosen_edges):
            chosen_edges.add((repo, t))
            get_edges.append((repo, t))
            break
testnodes = random.sample(range(301103), 60000)  # 抽取测试集
with open('../graph/test_edgel_neg.edgelist', 'w') as test:
    with open('../graph/train_edgel_neg.edgelist', 'w') as train:
        s = set(testnodes)
        for i in range(301103):
            if i in s:
                test.write(str(get_edges[i][0]) + ' ' + str(get_edges[i][1]) + '\n')
            else:
                train.write(str(get_edges[i][0]) + ' ' + str(get_edges[i][1]) + '\n')

In [9]:
# 使用node2vec对train_edgel_con.edgelist进行图嵌入，得到train_node2vec_con.emb
# 使用Logistic Regression进行link prediction

from sklearn.linear_model import LogisticRegression

frTrain = open('../graph/train_edgel_con.edgelist')                                        #打开训练集
frTest = open('../graph/test_edgel_con.edgelist')                                                #打开测试集
trainingSet = []; trainingLabels = []
testSet = []; testLabels = []
nodeemb = {}
nodenum = []
with open('../embedded/train_node2vec_con.emb', 'r') as emb:
    emb.readline()
    for line in emb.readlines():
        cl = line.split()
        m = int(cl[0])
        cl = cl[1:]
        cl = [float(x) for x in cl]
        nodeemb[m] = cl[:]
        nodenum.append(m)

for line in frTrain.readlines():
    [x, y] = line.split()
    x = int(x)
    y = int(y)
    lineArr = np.array(nodeemb[x]) * np.array(nodeemb[y])  # 边的特征
    trainingSet.append(lineArr)
    trainingLabels.append(1)
with open('../graph/train_edgel_neg.edgelist', 'r') as trainneg:
    for line in trainneg:
        [x, y] = line.split()
        x = int(x)
        y = int(y)
        lineArr = np.array(nodeemb[x]) * np.array(nodeemb[y])
        trainingSet.append(lineArr)
        trainingLabels.append(0)

for line in frTest.readlines():
    [x, y] = line.split()
    x = int(x)
    y = int(y)
    lineArr = np.array(nodeemb[x]) * np.array(nodeemb[y])
    testSet.append(lineArr)
    testLabels.append(1)
with open('../graph/test_edgel_neg.edgelist', 'r') as testneg:
    for line in testneg:
        [x, y] = line.split()
        x = int(x)
        y = int(y)
        lineArr = np.array(nodeemb[x]) * np.array(nodeemb[y])
        testSet.append(lineArr)
        testLabels.append(0)
i = 0
trainingSet = np.array(trainingSet)
trainingLabels = np.array(trainingLabels)
classifier = LogisticRegression(solver='liblinear',max_iter=10).fit(trainingSet, trainingLabels)
test_accurcy = classifier.score(testSet, testLabels) * 100
print('正确率:%f%%' % test_accurcy)
result = classifier.predict(testSet).tolist()
with open('../graph/prediction_hadamard_con.txt', 'w') as pred:
    for x in result:
        pred.write(str(x) + ' ')

正确率:89.519167%


In [11]:
with open('../graph/prediction_hadamard_con.txt', 'r') as prediction:
    p = prediction.readline()
    p = p.split()
    p = [int(x) for x in p]
tp = 0
fp = 0
fn = 0
for i in range(60000):
    if p[i]:
        tp += 1
    else:
        fn += 1
for i in range(60000, 120000):
    if p[i]:
        fp += 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
print(precision, recall, f1)

0.9458808928336373 0.83835 0.8888751446823174
